# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

##### 14th June 2020

### -- part 3: Explore and cluster the neighborhoods in Toronto

#### Preparing API credentials and downloading map json file

In [ ]:
# assign Foursquare credentials and required parameters
CLIENT_ID = 'Your_client_id' # your Foursquare ID
CLIENT_SECRET = 'Your_client_secret' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
# load Toronto json file
!wget -q -O 'Toronto.json' https://raw.githubusercontent.com/adamw523/toronto-geojson/master/simple.geojson
import json
print('Data downloaded!')

In [ ]:
with open('Toronto.json') as json_data:
    Toronto_data = json.load(json_data)

In [ ]:
Toronto_data

#### Use coordiante data to create dataframe

In [ ]:
# filter df
neighborhoods = df_latlng[['Postal Code', 'Borough', 'Latitude', 'Longitude']]

In [ ]:
neighborhoods.head()

In [ ]:
print('The dataframe has {} Postal code and {} Borough.'.format(
        len(neighborhoods['Postal Code'].unique()), len(neighborhoods['Borough'].unique())
)
     )

#### Prepare to plot a map using folium

In [ ]:
from geopy.geocoders import Nominatim
import folium

In [ ]:
# assign paramters for map plotting
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

#### Create a map with markers of each Postal Code

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
i=0
# add markers to map
for lat, lng, postcode, borough in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Postal Code'], 
                                            neighborhoods['Borough']):
    i += 1
    label = '{}, {}'.format(postcode,borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

print(i)
map_toronto

#### Explore neraby venues of each postal code

In [ ]:
# create a definition for retrieving venues in postal codes using coordinate values
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        result = requests.get(url).json()
        results = result["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                             'Neighborhood Latitude', 
                             'Neighborhood Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return (nearby_venues)

In [ ]:
Toronto_venues = getNearbyVenues(names=neighborhoods['Postal Code'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Toronto_venues

In [ ]:
Toronto_venues.head()

##### Check how many unique venue categories 

In [ ]:
print('{} venues were returned by Foursquare with {} different categories.'.format(
    len(Toronto_venues['Venue'].unique()),len(Toronto_venues['Venue Category'].unique())))

##### Check how many venues found in each postal code

In [ ]:
Toronto_venues.groupby('Neighborhood').count()

##### Create dummies for each unique venue category 

In [ ]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

Toronto_onehot

In [ ]:
# Add Postal code coulmn back into the dataframe
Toronto_onehot['Postal Code'] = Toronto_venues['Neighborhood']

In [ ]:
# Move the Postal code column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

In [ ]:
# Normalize venue categories in grouping venues in each postal code
Toronto_grouped = Toronto_onehot.groupby('Postal Code').mean().reset_index()
Toronto_grouped

##### Preview top three venues in each postal code

In [ ]:
num_top_venues = 3

for hood in Toronto_grouped['Postal Code']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Postal Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

#### Now let's create the new dataframe and display the top 5 venues for each postal code.

In [ ]:
# create definition for retrieving top venues in each postal code
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postal Code'] = Toronto_grouped['Postal Code']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

#### Clustering Postal Code using most common venues

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [41]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df_latlng

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Postal Code'), on='Postal Code')

Toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Park,Fireworks Store,Food & Drink Shop,Yoga Studio,Department Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Grocery Store,Coffee Shop,Hockey Arena,Portuguese Restaurant,Electronics Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Coffee Shop,Park,Breakfast Spot,Bakery,Theater
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Clothing Store,Furniture / Home Store,Vietnamese Restaurant,Event Space,Miscellaneous Shop
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Discount Store,Sandwich Place


In [ ]:
neighborhoods_venues_sorted

#### Finally, let's visualize the resulting clusters

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [ ]:
Toronto_merged.dropna(axis=0, inplace=True)
Toronto_merged['Cluster Labels'].astype('int')

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters